# Load microorganism

In [29]:
# Add path to the model
import sys
sys.path.append("../")
import pandas as pd
import cobra
from microorganism import Microorganism

# Read E. coli model from cobrapy
ecoli = cobra.io.load_json_model("../data/original/e_coli_core.json")

# Create a microorganism object
cobra_model = Microorganism(cobra_model=ecoli)

### Create Metabolic Network

In [3]:
# add folder for imports
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm
from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data
from pathway import Pathway
from utils import create_compound, create_reaction
# suppres rdkit warnings
import rdkit
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')


# read data from csv
cpds = pd.read_csv('../data/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('../data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('../data/pairs_final_RPAIRS_pred.csv', index_col=0)
cofactors = pd.read_csv('../data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)
pathway = Pathway()

data = create_compound(data, cpds, cofactors)
data = create_reaction(data, rxns)

# CREATE GRAPH
graph.create_graph(data=data, pairs=pairs)

graph.calculate_edge_mol_weight(data)
graph.calculate_smiles_similarity(data)

# nodes: 7997 
# edges: 11783


100%|██████████| 11783/11783 [00:25<00:00, 458.26it/s]


#### Get KEGG source and target

In [4]:
# define target compound
butanol_KEGG = 'C06142'
oxoglatarate_KEGG = 'C00026'

# define source list
src_list = ['C00084', 'C00024', 'C00074', 'C00022', 'C00497', 'C00149', \
            'C00122', 'C00042', 'C00036', 'C00091', 'C00111', 'C00158']

#### Find metabolic pathways from src to trg

Find metabolic pathways from `2-oxoglatarate` to `n-Butanol`

In [5]:
pathway.initialize(oxoglatarate_KEGG, butanol_KEGG, graph)
pathway.get_pathway()
pathway.print_pathway(with_reactions=True)

Pathway from C00026 to C06142:
['C00026', 'C00024', 'C00332', 'C01144', 'C00877', 'C00136', 'C01412', 'C06142']
[['R02772'], ['R00238', 'R09527'], ['R01975', 'R01976'], ['R03026'], ['R01171', 'R01175', 'R09738', 'R11675'], ['R01172', 'R01173'], ['R03544', 'R03545']]
['C00026', 'C00024', 'C00332', 'C03561', 'C00877', 'C00136', 'C01412', 'C06142']
[['R02772'], ['R00238', 'R09527'], ['R01977'], ['R03027'], ['R01171', 'R01175', 'R09738', 'R11675'], ['R01172', 'R01173'], ['R03544', 'R03545']]
['C00026', 'C00024', 'C00332', 'C01144', 'C03561', 'C00877', 'C00136', 'C01412', 'C06142']
[['R02772'], ['R00238', 'R09527'], ['R01975', 'R01976'], ['R03276'], ['R03027'], ['R01171', 'R01175', 'R09738', 'R11675'], ['R01172', 'R01173'], ['R03544', 'R03545']]
['C00026', 'C00024', 'C00332', 'C05116', 'C00877', 'C00136', 'C01412', 'C06142']
[['R02772'], ['R00238', 'R09527'], ['R05576'], ['R05595'], ['R01171', 'R01175', 'R09738', 'R11675'], ['R01172', 'R01173'], ['R03544', 'R03545']]
['C00026', 'C00024', 'C

In [59]:
p = pathway.pred_paths[pathway.idx_com[0]]
rxns = pathway.get_pathway_reactions(p)

# Select one from multiple reactions
for r in rxns:
    if len(r) > 1:
        for r_i in r:
            print(r_i)
            reactants = data.reactions[r_i].compounds[0]
            

        break

R00238
R09527


False

In [24]:
for r in rxns:
    for r_i in r:
        eq = data.reactions[r_i].equation
        print(r_i, eq)
    break

R02772 C06018 + C00026 + C00010 <=> C11907 + C00025 + C00024


In [35]:
# In a string map some words to others using a df
def str_map(string, df):
    for i in range(len(df)):
        string = string.replace(df.iloc[i, 1], df.iloc[i, 0])
    return string

s = str_map(eq, cobra_model.metabolites_df)
s

'C06018 + akg_c + coa_c <=> C11907 + glu__L_c + accoa_c'

In [42]:
cobra_model.metabolites_df[cobra_model.metabolites_df['kegg'] == 'C06018']

,metabolites,kegg,compartment


### Find pathways for all source compounds

In [ ]:
for src in src_list[:11]:
    pathway.initialize(src, butanol_KEGG, graph)
    pathway.get_pathway()
    pathway.print_pathway(with_reactions=True, to_bigg=True)    
    print('--------------------------------------')    